In [1]:
# Install required packages
!pip3 install beautifulsoup4
!pip3 install requests

import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

print("✅ Libraries installed and imported successfully")


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Libraries installed and imported successfully


In [2]:
# Define all helper functions
def date_time(table_cells):
    """
    This function returns the data and time from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML table cell 
    Input: the element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

print("✅ Helper functions defined successfully")

✅ Helper functions defined successfully


In [3]:
# Wikipedia page URL (snapshot from June 9th, 2021)
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

print("✅ URL and headers configured")

✅ URL and headers configured


In [4]:
print("🛰️  Starting SpaceX Wikipedia data collection...")

# Use requests.get() method with the provided static_url and headers
response = requests.get(static_url, headers=headers)

print(f"📡 API Response Status: {response.status_code}")

# Use BeautifulSoup() to create a BeautifulSoup object from response text content
soup = BeautifulSoup(response.content, 'html.parser')

# Use soup.title attribute to verify the page
print(f"📄 Page Title: {soup.title.string}")

print("✅ Wikipedia page loaded successfully")

🛰️  Starting SpaceX Wikipedia data collection...
📡 API Response Status: 200
📄 Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
✅ Wikipedia page loaded successfully


In [5]:
print("\n" + "="*50)
print("🔍 Extracting column names from HTML table...")

# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

print(f"📊 Found {len(html_tables)} tables on the page")

# Starting from the third table is our target table
first_launch_table = html_tables[2]
print("\nThird table preview:")
print(first_launch_table.prettify()[:1000] + "...")  # Show first 1000 characters

# Extract column names
column_names = []

# Apply find_all() function with `th` element on first_launch_table
th_elements = first_launch_table.find_all('th')

# Iterate each th element and apply the provided extract_column_from_header() to get a column name
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print("\n✅ Extracted column names:")
print(column_names)


🔍 Extracting column names from HTML table...
📊 Found 25 tables on the page

Third table preview:
<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span 

In [6]:
print("\n" + "="*50)
print("📝 Initializing launch data dictionary...")

launch_dict = dict.fromkeys(column_names)

# Remove an irrelevant column
del launch_dict['Date and time ( )']

# Initialize the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

print("✅ Launch dictionary initialized with columns:")
print(list(launch_dict.keys()))


📝 Initializing launch data dictionary...
✅ Launch dictionary initialized with columns:
['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time']


In [7]:
print("\n" + "="*50)
print("🚀 Parsing launch data from HTML tables...")

extracted_row = 0

# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    print(f"📋 Processing table {table_number + 1}...")
    
    # get table row 
    for rows in table.find_all("tr"):
        # check to see if first table heading is a number corresponding to launch number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
            
        # get table element 
        row = rows.find_all('td')
        
        # if it is number save cells in a dictionary 
        if flag:
            extracted_row += 1
            if extracted_row % 10 == 0:  # Progress indicator
                print(f"  Processed {extracted_row} rows...")
            
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time values
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            
            # Booster version
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string if row[1].a else None
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

print(f"✅ Successfully parsed {extracted_row} launch records")


🚀 Parsing launch data from HTML tables...
📋 Processing table 1...
📋 Processing table 2...
  Processed 10 rows...
📋 Processing table 3...
  Processed 20 rows...
📋 Processing table 4...
📋 Processing table 5...
  Processed 30 rows...
  Processed 40 rows...
📋 Processing table 6...
  Processed 50 rows...
  Processed 60 rows...
📋 Processing table 7...
  Processed 70 rows...
📋 Processing table 8...
  Processed 80 rows...
  Processed 90 rows...
  Processed 100 rows...
📋 Processing table 9...
  Processed 110 rows...
  Processed 120 rows...
✅ Successfully parsed 121 launch records


In [8]:
print("\n" + "="*50)
print("📊 Creating DataFrame from parsed data...")

# Create DataFrame from launch_dict
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

print(f"📈 DataFrame created with shape: {df.shape}")
print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nDataset information:")
print(df.info())


📊 Creating DataFrame from parsed data...
📈 DataFrame created with shape: (121, 11)

First 5 rows of the dataset:
  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          2       CCAFS                                Dragon            0   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome   Version Booster Booster landing  \
0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.18         Failure   
2   LEO     NASA        Success  F9 v1.07B0005.18    No attempt\n   
3   LEO     NASA      Success\n  F9 v1.07B0006.18      No attempt   
4   LEO     NASA      Success\n  F9 v1.07B0007.18    No attempt\n 

In [9]:
print("\n" + "="*50)
print("🔍 Checking data quality...")

print("\nMissing values in each column:")
print(df.isnull().sum())

print("\nData types:")
print(df.dtypes)


🔍 Checking data quality...

Missing values in each column:
Flight No.         0
Launch site        0
Payload            0
Payload mass       0
Orbit              0
Customer           2
Launch outcome     0
Version Booster    0
Booster landing    0
Date               0
Time               0
dtype: int64

Data types:
Flight No.         object
Launch site        object
Payload            object
Payload mass       object
Orbit              object
Customer           object
Launch outcome     object
Version Booster    object
Booster landing    object
Date               object
Time               object
dtype: object


In [10]:
print("\n" + "="*50)
print("💾 Exporting data to CSV file...")

# Export to CSV
df.to_csv('spacex_web_scraped.csv', index=False)

print("✅ Data successfully exported to 'spacex_web_scraped.csv'")

print("\n" + "="*50)
print("🎉 WEB SCRAPING COMPLETED SUCCESSFULLY!")
print(f"📁 File saved: spacex_web_scraped.csv")
print(f"📊 Dataset shape: {df.shape}")
print(f"🚀 Total launches scraped: {len(df)}")


💾 Exporting data to CSV file...
✅ Data successfully exported to 'spacex_web_scraped.csv'

🎉 WEB SCRAPING COMPLETED SUCCESSFULLY!
📁 File saved: spacex_web_scraped.csv
📊 Dataset shape: (121, 11)
🚀 Total launches scraped: 121


In [11]:
# Display final dataset summary
print("\n📋 Final Dataset Summary:")
print("=" * 50)

print(f"Total records: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

print("\nColumn overview:")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"  {col}: {unique_count} unique values")

print("\nFirst 3 launches:")
print(df.head(3).to_string())


📋 Final Dataset Summary:
Total records: 121
Total columns: 11
Date range: 1 March 2013 to 9 October 2017

Column overview:
  Flight No.: 121 unique values
  Launch site: 5 unique values
  Payload: 82 unique values
  Payload mass: 84 unique values
  Orbit: 8 unique values
  Customer: 37 unique values
  Launch outcome: 3 unique values
  Version Booster: 55 unique values
  Booster landing: 9 unique values
  Date: 121 unique values
  Time: 116 unique values

First 3 launches:
  Flight No. Launch site                               Payload Payload mass Orbit Customer Launch outcome   Version Booster Booster landing             Date   Time
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure      4 June 2010  18:45
1          2       CCAFS                                Dragon            0   LEO     NASA        Success  F9 v1.07B0004.18         Failure  8 December 2010  15:43
2          3       CCAFS      